In [1]:
import numpy as np
import pandas as pd

In [2]:
file = pd.read_csv("https://raw.githubusercontent.com/jehee1122/MLC-Final_foot-traffic-prediction/main/data/traffic_count.csv")
file.head()

,Unnamed: 0,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,...,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
0,0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,...,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0
1,1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,...,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0
2,2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,...,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0
3,3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,...,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0
4,4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,...,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0


In [3]:
import geopandas as gpd

In [4]:
geo = gpd.read_file("/Users/smritimohta/Desktop/MLC/Final Project/lion/lion.shp")
geo.head()

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,FromLeft,ToLeft,FromRight,ToRight,Join_ID,L_PD_Servi,R_PD_Servi,TRUCK_ROUT,SHAPE_Leng,geometry
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,599,699,596,716,2251001000000,None,None,None,396.030947,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,...,58,98,63,99,2798401000000,None,None,None,279.360514,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,...,1017,1079,1016,1084,2728001000000,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,4,UNION AVENUE,BEHAGEN PLAYGROUND COMFORT STA,0,U,None,B,None,W,DOT,...,0,0,0,0,21279502000000X,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,5,UNION AVENUE,BEHAGEN PLAYGROUND FIELD NORTH,0,U,None,B,None,W,DOT,...,0,0,0,0,21279503000000X,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [5]:
geo['SegmentID'] = geo['SegmentID'].apply(int)

In [6]:
geo1 = geo[['SegmentID','LZip','RZip','LBoro','RBoro','StreetWidt','StreetWi_1','StreetWi_2','BikeLane','TRUCK_ROUT','SHAPE_Leng','geometry']]
geo1.head()

,SegmentID,LZip,RZip,LBoro,RBoro,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,78126,10456,10456,2,2,34.0,34.0,None,None,None,396.030947,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,79796,10468,10468,2,2,30.0,30.0,None,None,None,279.360514,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [19]:
#converting crs of zipcode file to match crs of crime data file
geo1['geometry'] = geo1['geometry'].to_crs(epsg=4326)

/Users/smritimohta/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [24]:
alldata = file.merge(geo1, left_on = 'Segment ID', right_on ='SegmentID')

In [25]:
alldata.head()

,Unnamed: 0,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,...,RZip,LBoro,RBoro,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [13]:
alldata['StreetWi_2'].isnull().shape

(39575,)

In [15]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39575 entries, 0 to 39574
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Unnamed: 0     39575 non-null  int64   
 1   ID             39575 non-null  int64   
 2   Segment ID     39575 non-null  int64   
 3   Roadway Name   39575 non-null  object  
 4   From           39575 non-null  object  
 5   To             39575 non-null  object  
 6   Direction      39575 non-null  object  
 7   Date           39575 non-null  object  
 8   12:00-1:00 AM  39574 non-null  float64 
 9   1:00-2:00AM    39574 non-null  float64 
 10  2:00-3:00AM    39574 non-null  float64 
 11  3:00-4:00AM    39574 non-null  float64 
 12  4:00-5:00AM    39574 non-null  float64 
 13  5:00-6:00AM    39574 non-null  float64 
 14  6:00-7:00AM    39574 non-null  float64 
 15  7:00-8:00AM    39574 non-null  float64 
 16  8:00-9:00AM    39574 non-null  float64 
 17  9:00-10:00AM   39574 non-null  

In [26]:
alldata.drop(columns=['Unnamed: 0','ID','SegmentID','RZip','RBoro', 'StreetWi_2'], axis=1, inplace=True)
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,11.0,14.0,...,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,8.0,6.0,...,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,6.0,5.0,...,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,7.0,8.0,...,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,7.0,5.0,...,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [27]:
alldata['Roadway Name'] = alldata['Roadway Name'].str.title()
alldata['From'] = alldata['From'].str.title()
alldata['To'] = alldata['To'].str.title()

In [28]:
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Street','St')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Avenue','Ave')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Boulevard','Blvd')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('West','W')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('East','E')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Place','Pl')

In [29]:
alldata['From'] = alldata['From'].str.replace('street','st')
alldata['From'] = alldata['From'].str.replace('avenue','ave')
alldata['From'] = alldata['From'].str.replace('boulevard','blvd')
alldata['From'] = alldata['From'].str.replace('West','W')
alldata['From'] = alldata['From'].str.replace('East','E')
alldata['From'] = alldata['From'].str.replace('Place','Pl')

In [30]:
alldata['To'] = alldata['To'].str.replace('street','st')
alldata['To'] = alldata['To'].str.replace('avenue','ave')
alldata['To'] = alldata['To'].str.replace('boulevard','blvd')
alldata['To'] = alldata['To'].str.replace('West','W')
alldata['To'] = alldata['To'].str.replace('East','E')
alldata['To'] = alldata['To'].str.replace('Place','Pl')

In [31]:
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,...,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,...,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,...,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,...,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,...,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [32]:
import datetime

In [33]:
alldata['datetime'] = pd.to_datetime(alldata['Date'])

In [36]:
alldata['DOW'] = alldata['datetime'].dt.day_name()
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday


In [38]:
alldata[alldata['DOW']=='Monday'].shape

(4440, 41)

In [39]:
alldata[alldata['DOW']=='Tuesday'].shape

(4698, 41)

In [40]:
alldata[alldata['DOW']=='Wednesday'].shape

(4713, 41)

In [41]:
alldata[alldata['DOW']=='Saturday'].shape

(8107, 41)

In [42]:
alldata[alldata['DOW']=='Sunday'].shape

(8103, 41)

In [50]:
pd.set_option('display.max_columns', None)
alldata.head(10)

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,12.0,16.0,34.0,75.0,69.0,71.0,67.0,70.0,90.0,89.0,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,11.0,12.0,33.0,75.0,89.0,66.0,70.0,60.0,105.0,103.0,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,13.0,28.0,29.0,68.0,84.0,64.0,83.0,89.0,88.0,113.0,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday
5,15540,Beach St,Union Pl,Van Duzer Street,NB,01/14/2012,42.0,27.0,21.0,18.0,21.0,13.0,17.0,18.0,46.0,53.0,29.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-14,Saturday
6,15540,Beach St,Union Pl,Van Duzer Street,SB,01/09/2012,27.0,12.0,12.0,4.0,22.0,27.0,66.0,154.0,155.0,138.0,105.0,124.0,140.0,120.0,165.0,197.0,152.0,174.0,128.0,95.0,87.0,73.0,57.0,42.0,2276.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday
7,15540,Beach St,Union Pl,Van Duzer Street,SB,01/10/2012,26.0,16.0,11.0,13.0,16.0,27.0,59.0,156.0,177.0,131.0,107.0,108.0,122.0,131.0,192.0,180.0,161.0,171.0,120.0,96.0,90.0,70.0,63.0,49.0,2292.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday
8,15540,Beach St,Union Pl,Van Duzer Street,SB,01/11/2012,32.0,16.0,8.0,9.0,15.0,26.0,63.0,169.0,178.0,148.0,139.0,131.0,126.0,137.0,178.0,194.0,168.0,160.0,143.0,114.0,78.0,64.0,49.0,45.0,2390.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday
9,15540,Beach St,Union Pl,Van Duzer Street,SB,01/12/2012,24.0,12.0,7.0,18.0,11.0,23.0,61.0,146.0,177.0,128.0,117.0,111.0,134.0,134.0,171.0,184.0,157.0,167.0,148.0,112.0,86.0,77.0,63.0,51.0,2319.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday


In [49]:
day = []
for i in alldata['DOW']:
    if (i == 'Monday') | (i == 'Tuesday') | (i == 'Wednesday') | (i == 'Thursday') | (i == 'Friday'):
        day.append('Weekday')
    else: 
        day.append('Weekend')
        
day

['Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'We

In [51]:
alldata['day'] = day

In [52]:
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW,day
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday,Weekday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday,Weekday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,12.0,16.0,34.0,75.0,69.0,71.0,67.0,70.0,90.0,89.0,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday,Weekday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,11.0,12.0,33.0,75.0,89.0,66.0,70.0,60.0,105.0,103.0,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday,Weekday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,13.0,28.0,29.0,68.0,84.0,64.0,83.0,89.0,88.0,113.0,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday,Weekday


In [54]:
alldata_grouped = alldata[['Segment ID','LZip','LBoro','Roadway Name','From','To','Direction','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','Direction','LBoro','Segment ID','LZip']).mean()


In [60]:
alldata_grouped = alldata_grouped.sort_values(by=['LBoro','Segment ID'])
alldata_grouped.head(20)

12:00-1:00 AM  \
Roadway Name From            To                    day     Direction LBoro Segment ID LZip                   
State St     Bridge Street   Battery Pl            Weekday NB        1     22995      10004         344.80   
                                                           SB        1     22995      10004         199.50   
                                                   Weekend NB        1     22995      10004         250.75   
                                                           SB        1     22995      10004         162.75   
Broadway     Beaver Street   Lexington Avenue Line Weekday SB        1     23057      10004         105.80   
                                                   Weekend SB        1     23057      10004         112.50   
                             Morris Street         Weekday SB        1     23057      10004         175.00   
                                                   Weekend SB        1     23057      10004         196.50   
Broadway     Beaver Street   Morris Street         Weekday SB        1     23057      10004         548.20   
                                                   Weekend SB        1     23057      10004         511.75   
Trinity Pl   Edger St        Rector Pl             Weekday NB        1     23061      10006         134.00   
                                                   Weekend NB        1     23061      10006         180.00   
Fulton St    8 Avenue Line   8 Avenue Line         Weekday WB        1     23344      10038         164.80   
                                                   Weekend WB        1     23344      10038         185.00   
             Dutch Street    Nassau Street         Weekday WB        1     23344      10038         115.40   
                                                   Weekend WB        1     23344      10038         135.75   
             Nassau Street   Dutch Street          Weekday WB        1     23344      10038          73.00   
                                                   Weekend WB        1     23344      10038         196.75   
Church St    Duane St        Reade St              Weekday NB        1     23396      10013         574.00   
                                                   Weekend NB        1     23396      10013         819.25   

                                                                                             1:00-2:00AM  \
Roadway Name From            To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street   Battery Pl            Weekday NB        1     22995      10004   190.900000   
                                                           SB        1     22995      10004   139.300000   
                                                   Weekend NB        1     22995      10004   276.500000   
                                                           SB        1     22995      10004   131.625000   
Broadway     Beaver Street   Lexington Avenue Line Weekday SB        1     23057      10004    77.800000   
                                                   Weekend SB        1     23057      10004   100.000000   
                             Morris Street         Weekday SB        1     23057      10004   122.800000   
                                                   Weekend SB        1     23057      10004   136.000000   
Broadway     Beaver Street   Morris Street         Weekday SB        1     23057      10004   364.000000   
                                                   Weekend SB        1     23057      10004   381.250000   
Trinity Pl   Edger St        Rector Pl             Weekday NB        1     23061      10006    92.333333   
                                                   Weekend NB        1     23061      10006   155.250000   
Fulton St    8 Avenue Line   8 Avenue Line         Weekday WB        1     23344      10038    81.800000   
                                                   W

In [61]:
alldata_grouped.shape

(6491, 25)

In [74]:
alldata_grouped_nd = alldata[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','LBoro','Segment ID','LZip'], as_index=False).mean()


In [75]:
alldata_grouped_nd = alldata_grouped_nd.sort_values(by=['LBoro','Segment ID'])
alldata_grouped_nd.head(20)

,Roadway Name,From,To,day,LBoro,Segment ID,LZip,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
3906,State St,Bridge Street,Battery Pl,Weekday,1,22995,10004,272.15,165.100000,103.200000,79.050000,73.700000,121.650000,211.050000,333.650000,439.250000,475.75,421.750,416.650000,409.550000,393.350000,388.9000,409.300000,377.2000,385.90,359.3000,333.1500,309.500000,311.250000,472.350000,385.300000,7648.000000
3907,State St,Bridge Street,Battery Pl,Weekend,1,22995,10004,206.75,204.062500,145.687500,116.125000,93.000000,95.750000,147.562500,174.312500,202.625000,252.25,297.625,328.687500,349.500000,347.875000,351.9375,365.937500,340.9375,344.50,325.8125,275.9375,257.062500,233.000000,212.312500,192.000000,5861.250000
1221,Broadway,Beaver Street,Lexington Avenue Line,Weekday,1,23057,10004,105.80,77.800000,43.200000,43.400000,51.800000,80.800000,110.400000,117.000000,127.200000,125.60,120.800,120.400000,104.600000,105.800000,102.2000,106.800000,94.0000,104.20,107.4000,135.2000,147.400000,134.200000,129.400000,128.000000,2523.400000
1222,Broadway,Beaver Street,Lexington Avenue Line,Weekend,1,23057,10004,112.50,100.000000,85.750000,65.500000,34.500000,43.000000,35.250000,98.000000,98.000000,89.00,116.250,131.500000,100.750000,103.500000,140.5000,122.000000,121.7500,110.00,114.7500,114.5000,153.750000,110.000000,101.000000,74.750000,2376.500000
1223,Broadway,Beaver Street,Morris Street,Weekday,1,23057,10004,175.00,122.800000,97.200000,76.800000,82.600000,122.200000,155.400000,191.800000,247.800000,296.20,249.200,275.400000,296.400000,284.000000,266.0000,269.800000,237.2000,267.40,244.8000,260.6000,255.200000,284.000000,226.400000,217.800000,5202.000000
1224,Broadway,Beaver Street,Morris Street,Weekend,1,23057,10004,196.50,136.000000,118.250000,99.000000,69.250000,81.750000,90.250000,117.250000,142.250000,157.75,165.000,191.750000,197.250000,189.000000,197.7500,198.250000,182.0000,174.25,202.5000,184.5000,174.000000,207.000000,195.250000,180.750000,3847.500000
1315,Broadway,Beaver Street,Morris Street,Weekday,1,23057,10004,548.20,364.000000,244.200000,209.200000,222.400000,308.000000,470.000000,654.800000,806.200000,854.80,848.400,839.600000,838.600000,858.200000,849.2000,872.000000,831.8000,888.00,803.8000,725.6000,705.800000,842.400000,664.400000,620.000000,15869.600000
1316,Broadway,Beaver Street,Morris Street,Weekend,1,23057,10004,511.75,381.250000,338.500000,302.000000,229.000000,215.250000,260.250000,332.500000,508.000000,656.50,663.750,608.750000,781.000000,777.250000,734.2500,686.000000,601.2500,542.00,573.7500,519.0000,536.000000,576.500000,492.000000,421.750000,12248.250000
3993,Trinity Pl,Edger St,Rector Pl,Weekday,1,23061,10006,134.00,92.333333,72.333333,64.166667,54.333333,108.333333,177.166667,231.166667,366.833333,394.00,421.000,448.333333,416.666667,398.166667,409.0000,415.666667,407.0000,395.00,351.0000,305.5000,235.333333,233.333333,208.666667,186.833333,6526.166667
3994,Trinity Pl,Edger St,Rector Pl,Weekend,1,23061,10006,180.00,155.250000,143.750000,121.750000,94.500000,68.000000,77.750000,135.500000,207.000000,263.50,277.500,327.500000,343.000000,333.000000,327.7500,325.000000,281.7500,255.25,246.7500,216.7500,197.750000,177.250000,164.500000,153.500000,5074.250000


In [64]:
alldata_grouped_nd.shape

(4411, 25)

In [82]:
alldata_grouped_nd.to_csv("clean_nodirection.csv")

In [76]:
alldata_grouped_seg = alldata[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Segment ID','day'], as_index=False).mean()


In [77]:
alldata_grouped_seg = alldata_grouped_seg.sort_values(by=['Segment ID'])
alldata_grouped_seg.head(20)

,Segment ID,day,LBoro,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
0,202,Weekend,5.0,305.000000,222.000000,137.000000,138.000000,114.000000,155.000000,283.000000,355.000000,485.000000,549.000000,550.000000,555.000000,621.000000,630.000000,665.000000,649.000000,671.000000,700.000000,658.000000,646.000000,707.000000,620.000000,540.000000,504.000000,11459.000000
1,646,Weekday,5.0,28.250000,25.375000,19.000000,16.375000,26.375000,51.875000,97.250000,134.250000,182.750000,211.500000,200.875000,236.125000,250.375000,271.125000,263.625000,293.875000,290.750000,225.875000,183.000000,152.625000,94.875000,72.000000,43.000000,34.875000,3406.000000
2,1416,Weekday,5.0,37.400000,19.500000,10.400000,11.600000,30.700000,79.000000,174.300000,328.700000,318.000000,214.500000,234.600000,252.800000,272.700000,295.200000,360.800000,371.800000,375.600000,401.500000,360.300000,292.600000,228.200000,167.000000,108.700000,74.800000,5020.700000
3,1416,Weekend,5.0,92.125000,56.250000,30.625000,24.625000,21.000000,22.750000,47.625000,78.000000,156.500000,223.125000,295.250000,344.875000,361.250000,364.750000,342.875000,320.250000,311.000000,292.750000,268.750000,223.250000,188.000000,142.375000,119.875000,89.125000,4417.000000
4,1421,Weekday,5.0,53.750000,30.250000,21.750000,19.000000,25.125000,55.000000,151.500000,315.500000,353.375000,213.000000,219.625000,259.250000,287.125000,294.125000,352.125000,400.375000,395.375000,410.375000,366.250000,315.625000,183.625000,140.125000,87.875000,61.750000,5011.875000
5,1883,Weekday,5.0,37.000000,20.916667,12.916667,6.583333,24.333333,66.250000,168.833333,387.833333,306.250000,206.916667,209.333333,223.333333,262.500000,282.166667,362.916667,376.916667,384.333333,375.250000,363.166667,293.250000,227.500000,168.500000,115.250000,86.750000,4969.000000
6,1883,Weekend,5.0,96.875000,64.625000,31.625000,22.000000,14.750000,21.625000,48.125000,98.375000,172.375000,248.625000,290.500000,347.500000,376.125000,340.625000,337.500000,324.875000,308.375000,295.625000,295.625000,247.750000,190.625000,154.250000,109.125000,92.250000,4529.750000
7,1884,Weekday,5.0,52.500000,28.500000,23.500000,9.000000,17.500000,53.500000,145.000000,395.000000,331.500000,206.000000,222.500000,242.500000,290.500000,314.000000,367.500000,442.000000,380.000000,405.500000,371.500000,312.000000,239.000000,170.500000,135.500000,88.000000,5243.000000
8,1885,Weekday,5.0,58.000000,31.000000,22.500000,16.500000,17.000000,45.000000,155.500000,378.500000,352.000000,197.500000,179.500000,228.000000,255.000000,268.000000,356.500000,387.000000,364.000000,327.500000,338.500000,286.000000,227.000000,171.500000,141.500000,105.000000,4908.500000
9,1886,Weekday,5.0,66.500000,35.500000,29.000000,23.500000,21.000000,42.000000,106.500000,382.500000,355.500000,205.000000,208.000000,240.500000,291.000000,315.000000,414.500000,430.000000,373.500000,377.500000,363.500000,333.000000,130.000000,0.000000,0.000000,0.000000,4743.500000


In [73]:
alldata_grouped_seg.shape

(3359, 26)

In [81]:
alldata_grouped_seg.to_csv("clean_seg.csv")